# Transformaciones necesarias para las FUNCIONES de la API

En este apartado se hicieron todos las tranformaciones necesarias para las funciones que va a consumir la API. Se tienen como fuentes de datos los archivos ya limpios: user_reviews_cleaned, steam_games_cleaned, user_items_cleaned. 

Importamos bibliotecas necesarias

In [1]:
import pandas as pd
import os
import ast

Cargar los archivos limpios

In [2]:
df_steam_games = pd.read_csv('steam_games_cleaned.csv')
df_user_reviews = pd.read_csv('user_reviews_cleaned.csv')
df_user_items = pd.read_csv('user_items_cleaned.csv')

Arreglos

1. - PlayTimeGenre

In [3]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_items, df_steam_games[['item_id', 'year', 'genres']], on='item_id', how='inner')

#Agrupamos por genero y año de lanzamiento y sumar las horas jugadas
#df_merged = df_merged.groupby(['genres', 'year']).agg({'hours_game': 'sum'}).reset_index()

#Agrupamos por genero y año de lanzamiento
df_merged = df_merged.groupby(['year', 'genres'])['hours_game'].sum().reset_index()

#Convertimos la columna genres a tipo lista
df_merged['genres'] = df_merged['genres'].apply(ast.literal_eval)

#Expandimos las listas en columnas
df_merged = df_merged.explode('genres').reset_index(drop=True)

# Encontrar el género con más horas jugadas por año 
max_hours_genre = df_merged.loc[df_merged.groupby(['year', 'genres'])['hours_game'].idxmax()]

# Ordenar el DataFrame resultante
max_hours_genre = max_hours_genre.sort_values(by=['genres', 'hours_game', 'year'], ascending=[True, False, True])

# Guardar el resultado en un archivo CSV
max_hours_genre.to_csv('PlayTimeGenre.csv', index=False)

2. - UserForGenre

In [4]:
df_game = df_steam_games.drop(['name', 'developer', 'price', 'tags','specs'],axis=1)

# Merge de los DataFrames
merged_data=pd.merge(df_user_items,df_game,left_on='item_id', right_on='item_id', how='right')

merged_data.drop(['item_id'], axis=1, inplace=True)
merged_data.dropna(inplace=True)

merged_data['hours_game'] = merged_data['hours_game'].astype(int)
merged_data['year'] = merged_data['year'].astype(int)

#Convertimos la columna genres a tipo lista
merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)

#Expandimos las listas en columnas
merged_data = merged_data.explode('genres').reset_index(drop=True)

#Encontrar el jugador con más horas jugadas por género y año
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'year'])['hours_game'].idxmax()]

#Filtrar las columnas necesarias
max_hours_df = max_hours_df[['genres', 'year', 'user_id', 'hours_game']]

#Segunda agrupación para obtener las horas jugadas por año para el usuario seleccionado
max_hours = max_hours_df.groupby(['genres', 'year', 'user_id'])['hours_game'].sum().reset_index()

max_hours.to_csv('UserForGenre.csv', index=False)

3. - UsersRecommend

In [5]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año
top3_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_by_year.to_csv('UsersRecommend.csv', index=False)

4. - UsersWorstDeveloper

In [6]:
# Unir los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
 
# Renombrar la columna 'year_x' a 'year'
df_filtered = df_filtered.rename(columns={'year_x': 'year'})

# Contar las ocurrencias de cada desarrolladora por año
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
    
# Ordenar por 'year' y 'count' en orden descendente
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    

# Obtener el top 3 por año
result_df = grouped_result.groupby('year').head(3)    

# Guardar el resultado en un archivo CSV
result_df.to_csv('UsersWorstDeveloper.csv', index=False)

5. - Sentiment_analysis

In [7]:
# Merge de los DataFrames
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_csv('sentiment_analysis.csv', index=False)